# LME and Creatinine

## Joining Data

In [50]:
library(data.table)
library(nlme)

creatinine.data <- data.table(read.csv('creat.csv'))
setkey(creatinine.data, 'group')
head(creatinine.data)

id,group,age,scr
1,1,35.765,0.182
1,1,37.990,0.088
3,1,51.083,0.156
3,1,52.386,0.116
3,1,52.805,0.087
3,1,52.997,0.067


In [51]:
group.data <- data.table(
    group=c(1,2,3,4),
    kidney.disease=c(1,1,0,0),
    hypertension=c(1,0,1,0)
)
setkey(group.data, 'group')
group.data

group,kidney.disease,hypertension
1,1,1
2,1,0
3,0,1
4,0,0


In [52]:
creatinine.group.data <- creatinine.data[group.data]
head(creatinine.group.data)

id,group,age,scr,kidney.disease,hypertension
1,1,35.765,0.182,1,1
1,1,37.990,0.088,1,1
3,1,51.083,0.156,1,1
3,1,52.386,0.116,1,1
3,1,52.805,0.087,1,1
3,1,52.997,0.067,1,1


## Fitting LME Model

In [90]:
scr.model <- lme(scr ~ age*kidney.disease,
                 random=reStruct(~age|id, pdClass='pdDiag'),
                 correlation=corExp(form=~age|id, nugget=TRUE),
                 method='ML', data=creatinine.group.data)
summary(scr.model)

Linear mixed-effects model fit by maximum likelihood
 Data: creatinine.group.data 
        AIC       BIC   logLik
  -127.9923 -79.67728 72.99617

Random effects:
 Formula: ~age | id
 Structure: Diagonal
        (Intercept)         age  Residual
StdDev:   0.1317441 0.004823218 0.2258346

Correlation Structure: Exponential spatial correlation
 Formula: ~age | id 
 Parameter estimate(s):
    range    nugget 
4.6701089 0.2299759 
Fixed effects: scr ~ age * kidney.disease 
                        Value  Std.Error  DF   t-value p-value
(Intercept)         1.1906761 0.05430051 964 21.927530  0.0000
age                -0.0031083 0.00143749 964 -2.162279  0.0308
kidney.disease      0.3135749 0.07146362 617  4.387896  0.0000
age:kidney.disease -0.0166492 0.00185944 964 -8.953872  0.0000
 Correlation: 
                   (Intr) age    kdny.d
age                -0.928              
kidney.disease     -0.760  0.705       
age:kidney.disease  0.718 -0.773 -0.935

Standardized Within-Group Residuals:

[1] 966

In [39]:
creatinine.data[,max(group),keyby=c('id')][,length(id),keyby=c('V1')]
#unique(creatinine.data[,'id'])

V1,V1
1,294
2,103
3,73
4,149


In [20]:
#creatinine.data[,list(count=length(id)),by=c('group')]

group,count
1,1001
2,202
4,240
3,142


In [18]:
nrow(creatinine.data)

[1] 1585